**Welcome to the Eco2AI Replication Project**


This notebook replicates selected results from the paper **"Eco2AI: A Framework for Energy Consumption and CO₂ Emission Tracking in AI Experiments"** (Budennyy et al., 2022).

The aim of this project was to reproduce key emission tracking outputs by training a simple Convolutional Neural Network (CNN) on the MNIST dataset while logging carbon emissions using the Eco2AI Python package.

**🛠 Setup instructions
If required, install dependencies:**




In [ ]:
!pip install numpy pandas matplotlib pillow torch torchvision eco2ai

In [ ]:
# Suppress Future Warnings for Cleaner Output
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:
# Import Libraries
import eco2ai
from eco2ai import track

In [ ]:
# Load Core Libraries for Model Development and Utilities
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from PIL import Image
import os

In [ ]:
# Prepare and Load MNSIT Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
train_dataset

In [ ]:
# Visualise Sample Images
fig, axs = plt.subplots(1, 10, figsize=(25, 3))
for i in range(10):
    label_indexes = [idx for idx, (_, label) in enumerate(train_dataset) if label == i]
    index = random.choice(label_indexes)
    img, _ = train_dataset[index]
    axs[i].imshow(img.squeeze(), cmap='gray')
plt.show()

In [ ]:
# Create Data Loaders to Prepare Batches for Training and Testing
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Define the CNN  Model Arhitecture
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.batch_norm = nn.BatchNorm2d(1)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.25)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * 14 * 14, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.batch_norm(x)
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [ ]:
# Model Training and Evaluation Loop

# Train the model and evaluate accuracy on the validation set
def train(model, loader, criterion, optimizer, epochs=3):
    history = {'train_acc': [], 'val_acc': []}
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for epoch in range(epochs):
        model.train() # Set model to training mode
        correct, total = 0, 0 # Track correct predictions and total samples

        # Training Loop
        for x, y in loader:
            x, y = x.to(device), y.to(device) # Move data to GPU or CPU
            optimizer.zero_grad() # Clear previous gradients
            outputs = model(x) # Forward pass: generate predictions
            loss = criterion(outputs, y) # Compute loss against true labels
            loss.backward() # Backpropagate gradients
            optimizer.step() # Update weights


            _, predicted = outputs.max(1) # Get class with highest score
            total += y.size(0)
            correct += (predicted == y).sum().item()

        train_acc = correct / total  # Compute accuracy
        history['train_acc'].append(train_acc)

        # Validation Loop
        model.eval() # Set model to evaluation mode
        correct, total = 0, 0
        with torch.no_grad(): # Disable gradient tracking during evaluation
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                outputs = model(x)
                _, predicted = outputs.max(1)
                total += y.size(0)
                correct += (predicted == y).sum().item()

        val_acc = correct / total # Compute validation accuracy
        history['val_acc'].append(val_acc)

        # Print summary after each epoch
        print(f"Epoch {epoch + 1}/{epochs}, Train Accuracy: {train_acc:.4f}, Val Accuracy: {val_acc:.4f}")

    return history # Return accuracy results for later use


In [ ]:
# Begin Emissions Tracking and Train Model

# Emissions Tracking - Run 1 (3 Epochs, Batch Size = 256)
tracker = eco2ai.Tracker(project_name="mnist_replication", experiment_description="3 epochs, batch size 256", file_name="emission.csv")

tracker.start() # Begin tracking energy usage and emissions

# Define data loaders with a batch size of 256
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# Set up model, loss fucntion and optimiser
model = SimpleCNN().to("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model for 3 epochs and collect accuracy stats
history = train(model, train_loader, criterion, optimizer, epochs=3)

tracker.stop() # End emissions tracking


In [ ]:
# Emissions Tracking – Run 2 (2 Epochs, Batch Size = 128)

# Trains the same CNN using a smaller batch size (128) and fewer epochs (2)
# Useful for comparing how batch size and training time affect emissions and accuracy

tracker = eco2ai.Tracker(project_name="mnist_replication", experiment_description="2 epochs, batch size 128", file_name="emission.csv")
tracker.start()

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

model = SimpleCNN().to("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.Adam(model.parameters(), lr=0.001)

history = train(model, train_loader, criterion, optimizer, epochs=2)

tracker.stop()


In [ ]:
# Emissions Tracking – Run 3 (4 Epochs, Batch Size = 64)
tracker = eco2ai.Tracker(project_name="mnist_replication", experiment_description="4 epochs, batch size 64, SGD", file_name="emission.csv")
tracker.start()

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model = SimpleCNN().to("cuda" if torch.cuda.is_available() else "cpu")
optimizer = optim.SGD(model.parameters(), lr=0.01)

history = train(model, train_loader, criterion, optimizer, epochs=4)

tracker.stop()


In [ ]:
# Load the emissions CSV and display the last 3 tracked runs
df = pd.read_csv("emission.csv")
display(df.tail(3))